In [238]:
import pandas as pd
import numpy as np
from functools import partial
import re

In [239]:
def cross_movment(trace_string):
    bad_for_SLI = False
    for i in re.findall('\*T\*-(\d+)', trace_string):
        pos_end = re.search(f'\([A-Z]+-{i}', trace_string).span()[1]
        T_start = re.search(f'\*T\*-{i}', trace_string).span()[0]
        counter =1
        if pos_end<T_start:
            for j in range(pos_end, T_start):
                if trace_string[j] == ')':
                    counter -=1
                if trace_string[j] == '(':
                    counter +=1
                if counter ==0:
                    pos_end =j
                    break
        start_search, end_search  = min(pos_end, T_start), max(pos_end, T_start)
        if len(re.findall('[a-z]+', trace_string[start_search: end_search])) >= 1:
            bad_for_SLI = True
    return bad_for_SLI


In [243]:
with open(r'C:\Users\shani\Downloads\wiki-simplified.parser.ptb', 'rb') as f:
    parenthsis_counter = 0
    sentences = []
    sentence = ''
    for line in f:
        if chr(line[0]) == '(':
            sentences.append((str(sentence).replace('\\n','').replace("b'", "").replace(")'",")")))
            sentence=''
        sentence += str(line)
    sentences.append(str(sentence))
    
df = pd.DataFrame(sentences, columns=['TreeString'])
df['isTrace'] = df.TreeString.str.find('*T*') > 0
df['isQuestion'] = df.TreeString.str.find('. ?') > 0
df['isCrossMovment'] = df.TreeString.apply(cross_movment)

In [244]:
df.to_csv('wiki_simplified_processed.csv')